# ***Image Classification for Multiple classes using Convolutional Neural Networks***


USES : CAN BE USED FOR AUTOMATED IDENTIFICATION OF THE NATURAL DISASTERS AND CALAMITIES VIA LIVE IMAGES AND SEND THE PRIOTITY CALL TO RESCUE TEAM.

In [ ]:


import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
from keras.optimizers import RMSprop
from keras.preprocessing import image
from keras.utils import to_categorical
from keras.models import load_model
from PIL import Image
import cv2

# Data loading
train_data_dir = '#training image path'
test_data_dir = '#test image path'
train_image_size = 150
test_image_size = 150

# Data augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

test_datagen = ImageDataGenerator(rescale=1./255)

# Load training data
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(train_image_size, train_image_size),
    batch_size=32,
    class_mode='categorical')

# Load testing data
test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(test_image_size, test_image_size),
    batch_size=32,
    class_mode='categorical')

# Model architecture
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(train_image_size, train_image_size, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(5))
model.add(Activation('softmax'))

# Compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train model
model.fit_generator(
    train_generator,
    steps_per_epoch=13,
    epochs=10,
    validation_data=test_generator,
    validation_steps=50)

# Save model
model.save('5class_image_classification_model.h5')

# Load model
model = load_model('5class_image_classification_model.h5')

# Predict on new image
image_path = '#Exceution image path'
img = image.load_img(image_path, target_size=(test_image_size, test_image_size))
x = image.img_to_array(img)
x = x.reshape((1,) + x.shape)
x /= 255.0
pred = model.predict(x)
pred_class = np.argmax(pred)
print(pred_class)

# Get class name
class_names = ['Earthquakes', 'Flood', 'Fire', 'Tsunamis', 'Snow Storm']
print(class_names[pred_class])

# Increase resolution of input image
def increase_resolution(image_path, scale_factor):
  img = cv2.imread(image_path)
  height, width, channels = img.shape
  new_height = int(height * scale_factor)
  new_width = int(width * scale_factor)
  img = cv2.resize(img, (new_width, new_height), interpolation=cv2.INTER_LINEAR)
  cv2.imwrite(image_path, img)

# Increase resolution of test image
increase_resolution(image_path, 2)

# Predict on new image with increased resolution
img = image.load_img(image_path, target_size=(test_image_size, test_image_size))
x = image.img_to_array(img)
x = x.reshape((1,) + x.shape)
x /= 255.0
pred = model.predict(x)
pred_class = np.argmax(pred)
print(pred_class)

# Get class name
class_names = ['Earthquakes', 'Flood', 'Fire', 'Tsunamis', 'Snow Storm']
print(class_names[pred_class])
